In [60]:
"""
This code converts .dly files from the ghcn daily database to a csv format, after selecting a single metric and splitting the daily data into separate rows for machine learning.
John Moen

Dataset Database: https://www.ncei.noaa.gov/pub/data/ghcn/daily/ (dwnload link: https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd_all.tar.gz)
Dataset Readme: https://www.ncei.noaa.gov/pub/data/ghcn/daily/readme.txt
"""

import pandas as pd

METRIC = "TMAX"
STATION = "ACW00011604.dly"
FILE_PATH = f"../data/raw_station_data/{STATION}"
OUT_FILE_PATH = f"../data/cleaned_station_data/{STATION[:-4]}.csv"

In [61]:
"""
  Generate the colspecs, these are for the .fwf 'fixed width format' filetypes. 
  The .read_fwf()'s  'infer' does not work here as some data is left blank as a value
  
  From the data set documentation:

    ------------------------------
    Variable   Columns   Type
    ------------------------------
    ID            1-11   Character
    YEAR         12-15   Integer
    MONTH        16-17   Integer
    ELEMENT      18-21   Character
    VALUE1       22-26   Integer
    MFLAG1       27-27   Character
    QFLAG1       28-28   Character
    SFLAG1       29-29   Character
    VALUE2       30-34   Integer
    MFLAG2       35-35   Character
    QFLAG2       36-36   Character
    SFLAG2       37-37   Character
      .           .          .
      .           .          .
      .           .          .
    VALUE31    262-266   Integer
    MFLAG31    267-267   Character
    QFLAG31    268-268   Character
    SFLAG31    269-269   Character
    ------------------------------

  I generate a list of of tuples each representing the 'columns' column above, skipping the M,Q, and S flags
"""

# Initialize colspecs with ID, YEAR, MONTH, and ELEMENT
colspecs = [(0, 11), (11, 15), (15, 17), (17, 21)]


# Generate remaining colspecs for each (31) day, skipping the flags
x = 21
for i in range(31):
    colspecs.append((x,x+5))
    x += 8



[(0, 11), (11, 15), (15, 17), (17, 21), (21, 26), (29, 34), (37, 42), (45, 50), (53, 58), (61, 66), (69, 74), (77, 82), (85, 90), (93, 98), (101, 106), (109, 114), (117, 122), (125, 130), (133, 138), (141, 146), (149, 154), (157, 162), (165, 170), (173, 178), (181, 186), (189, 194), (197, 202), (205, 210), (213, 218), (221, 226), (229, 234), (237, 242), (245, 250), (253, 258), (261, 266)]


In [62]:
# Read in the data
df = pd.read_fwf(FILE_PATH, header=None, colspecs=colspecs)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,ACW00011604,1949,1,TMAX,289,289,283,283,289,289,...,272,267,267,267,278,272,272,272,272,272
1,ACW00011604,1949,1,TMIN,217,228,222,233,222,222,...,217,211,211,200,222,217,211,222,206,217
2,ACW00011604,1949,1,PRCP,0,30,0,0,25,41,...,0,53,0,10,15,41,0,86,28,15
3,ACW00011604,1949,1,SNOW,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACW00011604,1949,1,SNWD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
"""
Convert the data into a machine learning usable format, w/ each day on a different row

"""

# Initialize the Data frame
columns = ["station_id", "year", "month", "day", "metric", "value"]
new_df = pd.DataFrame(columns=columns)

# Loop through raw dataframe, only pulling the chosen METRIC
for index, row in df.iterrows():
    if row[3] == "TMAX":
        day_counter = 4

        # For each day in the month, create a row
        for i in range(30):
            new_df.loc[len(new_df.index)] = [row[0], row[1], row[2], day_counter - 3, row[3], row[day_counter] / 10]
            day_counter += 1

new_df.head()


,station_id,year,month,day,metric,value
0,ACW00011604,1949,1,1,TMAX,28.9
1,ACW00011604,1949,1,2,TMAX,28.9
2,ACW00011604,1949,1,3,TMAX,28.3
3,ACW00011604,1949,1,4,TMAX,28.3
4,ACW00011604,1949,1,5,TMAX,28.9


In [69]:
# Save the dataframe to a csv with the same filename
new_df.to_csv(,index=False)